Tests for retrieving data from PACS
--

Imports
--

In [ ]:
import logging
import configparser
from datetime import datetime as dt

from IPython.core import display as ICD

from pydicom.dataset import Dataset

from scripts.run_all import run_all
from scripts.retrieve_data_from_PACS import find_data, get_data

#from pynetdicom import debug_logger
#debug_logger()

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

Set the logger of the pynetdicom module to ERROR, to avoid logs

In [ ]:
logging.getLogger('pynetdicom').setLevel(logging.ERROR)

Initialize the "config" object

In [ ]:
config = run_all()
#df_study = retrieve_studies(config)
#df_study

Get (C-MOVE) a CT series first and last frame

In [ ]:
# Create out identifier (query) dataset
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.SeriesInstanceUID = '1.2.840.113619.2.80.3834766814.478.1569931317.104.4.1'
query_ds.StudyInstanceUID = '1.2.826.0.1.3680043.2.146.2.20.2560235.1900228608.0'
query_ds.PatientID = '2560235'
query_ds.StudyID = '40464'
query_ds.StudyDate  = '20191001'
query_ds.Modality = 'CT'
query_ds.InstanceNumber = ['1', '131']

df = get_data(config, query_ds, ['PatientID', 'ManufacturerModelName', 'InstanceNumber', 'StudyTime',
                                 'SeriesTime', 'ContentTime', 'AcquisitionTime'])
df

Get (C-MOVE) a PT series first and last frame

In [ ]:
# Create out identifier (query) dataset
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.SeriesInstanceUID = '1.3.12.2.1107.5.6.1.2013.31330119100111184984100000023'
query_ds.PatientID = '69168'
query_ds.StudyDate  = '20191001'
query_ds.Modality = 'PT'
query_ds.InstanceNumber = ['1', '387']

df = get_data(config, query_ds, ['PatientID', 'ManufacturerModelName', 'InstanceNumber', 'StudyTime',
                                 'SeriesTime', 'ContentTime', 'AcquisitionTime'])
df

In [ ]:
# Create out identifier (query) dataset
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.SeriesInstanceUID = '1.3.12.2.1107.5.1.4.10001.30000019100107233748400041397'
query_ds.PatientID = '1084649'
query_ds.StudyDate  = '20191001'
query_ds.Modality = 'PT'

query_ds.InstanceNumber = ['1', '1000', '2772']

df = get_data(config, query_ds, ['PatientID', 'ManufacturerModelName', 'InstanceNumber', 'StudyTime',
                                 'SeriesTime', 'ContentTime', 'AcquisitionTime'])
df

Get all the series corresponding to the query Dataset's filter [date = 2019-10-01, study time = between 8am and 9am]

In [ ]:
query_ds = Dataset()

# set the filtering parameters
query_ds.QueryRetrieveLevel = 'SERIES'
query_ds.StudyDate  = '20191001'
query_ds.SeriesDate = '20191001'
query_ds.StudyTime  = '080000-090000'
query_ds.SeriesTime = '080000-090000'

# parameters to fetch
query_ds.NumberOfSeriesRelatedInstances = ''
query_ds.SeriesInstanceUID = ''
query_ds.Modality = ''
query_ds.PatientID = ''
query_ds.StudyID = ''

logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

df_series_all = find_data(config, query_ds)
df_series_all

Get a list of Series having more than a few frames

In [ ]:
df_series = df_series_all[df_series_all['Number of Series Related Instances'].astype(int) > 2][0:3]
df_series

Get time differences by querying the ContentTime on the IMAGE level

In [ ]:
time_format = '%H%M%S'

for i in df_series.index:
    
    series_UID = df_series.loc[i, 'Series Instance UID']
    last_frame = df_series.loc[i, 'Number of Series Related Instances']
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = ['1', last_frame]
    query_ds.SeriesInstanceUID = series_UID
    query_ds.ContentTime = ''
    
    df = retrieve_data(config, query_ds)
    
    start_time = dt.strptime(df.loc[0, 'Content Time'], time_format)
    end_time = dt.strptime(df.loc[1, 'Content Time'], time_format)
    delta_sec = (end_time - start_time).seconds

    logging.info('Series total time: end [{:%H:%M:%S}] - [{:%H:%M:%S}] = {} seconds'.format(end_time, start_time, delta_sec))

In [ ]:
df_series_pt = \
    df_series_all[
        (df_series_all.Modality == 'PT')
      & (df_series_all['Number of Series Related Instances'].astype(int) > 1)
    ][0:10]
ICD.display(df_series_pt)
print(df_series_pt.loc[556, 'Series Instance UID'])
print(df_series_pt.loc[556, 'Patient ID'])

In [ ]:
time_format = '%H%M%S'

for i in df_series_pt.index:
    
    series_UID = df_series_pt.loc[i, 'Series Instance UID']
    last_frame = df_series_pt.loc[i, 'Number of Series Related Instances']
    mid_frame = str(int(int(last_frame) / 2))
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = ['1', mid_frame, last_frame]
    query_ds.SeriesInstanceUID = series_UID
    query_ds.StudyTime = ''
    query_ds.SeriesTime = ''
    query_ds.ContentTime = ''
    query_ds.NumberOfSeriesRelatedInstances = ''
    
    df = retrieve_data(config, query_ds)
    if len(df) > 0:
        ICD.display(df)

In [ ]:
df.loc[0, 'Content Time'].split('.')[0]

Get all PT images with InstanceNumber 1 for a specific day (20191001)

In [ ]:
query_ds = Dataset()

# set the filtering parameters
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.StudyDate  = '20191001'
query_ds.InstanceNumber = '1'
query_ds.Modality = 'PT'

# parameters to fetch
query_ds.StudyTime = ''
query_ds.SeriesTime = ''
query_ds.ContentTime = ''
query_ds.NumberOfSeriesRelatedInstances = ''
query_ds.SeriesInstanceUID = ''

logging.info('Query Dataset:')
for s in str(query_ds).split('\n'): logging.info('    ' + s)

df_PT_IN1_20191001_all = retrieve_data(config, query_ds)
df_PT_IN1_20191001_all

Get the content time of the last frame

In [ ]:
df_PT_IN1_20191001_copy = df_PT_IN1_20191001_all.copy()
for i in df_PT_IN1_20191001_copy.index:
    
    series_UID = df_PT_IN1_20191001_copy.loc[i, 'Series Instance UID']
    last_frame = df_PT_IN1_20191001_copy.loc[i, 'Number of Series Related Instances']
    mid_frame = str(int(int(last_frame) / 2))
    
    logging.info('Fetching info for index {}: "{}"'.format(i, series_UID))
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = last_frame
    query_ds.SeriesInstanceUID = series_UID
    query_ds.ContentTime = ''
    
    df = retrieve_data(config, query_ds)
    if len(df) > 0:
        df_PT_IN1_20191001_copy.loc[i, 'Last Content Time'] = df.loc[0, 'Content Time']
    
    query_ds = Dataset()
    query_ds.QueryRetrieveLevel = 'IMAGE'
    query_ds.InstanceNumber = mid_frame
    query_ds.SeriesInstanceUID = series_UID
    query_ds.ContentTime = ''
    
    df = retrieve_data(config, query_ds)
    if len(df) > 0:
        df_PT_IN1_20191001_copy.loc[i, 'Mid Content Time'] = df.loc[0, 'Content Time']

In [ ]:
df_PT_IN1_20191001_copy[['Content Time', 'Mid Content Time', 'Last Content Time', 'Instance Number', 'Number of Series Related Instances']]

In [ ]:
s = df_PT_IN1_20191001_copy.loc[0:39, 'Content Time'].apply(lambda s: s.split('.')[1]).astype(int)
e = df_PT_IN1_20191001_copy.loc[0:39, 'Last Content Time'].apply(lambda s: s.split('.')[1]).astype(int)
n = df_PT_IN1_20191001_copy.loc[0:39, 'Number of Series Related Instances'].astype(int)
d = e - s
import pandas as pd
a = pd.DataFrame([s, e, n, d, d/n]).transpose()
a.columns = ['s', 'e', 'n', 'd', 'd/n']
a[a['n'] > 400]

Try to use MOVE instead of FIND

In [ ]:
df_series.loc[0]['Series Instance UID']

In [ ]:
ds = Dataset()
ds.QueryRetrieveLevel = 'IMAGE'
ds.SeriesInstanceUID = '1.2.840.113619.2.80.3834766814.478.1569931317.104.4.1'
ds.StudyInstanceUID = '1.2.826.0.1.3680043.2.146.2.20.2560235.1900228608.0'
ds.PatientID = '2560235'
ds.StudyID = '40464'
ds.StudyDate  = '20191001'
ds.Modality = 'CT'
ds.InstanceNumber = '1'

df = retrieve_data(config, ds)
df

In [ ]:
from pydicom.dataset import Dataset

from pynetdicom import AE, evt, StoragePresentationContexts
from pynetdicom.sop_class import PatientRootQueryRetrieveInformationModelMove

datasets = []
def handle_store(event):
    """Handle a C-STORE service request"""
    print('HANDLE STORE')
    datasets.append(event.dataset)
    # Ignore the request and return Success
    return 0x0000

handlers = [(evt.EVT_C_STORE, handle_store)]

# Initialise the Application Entity
ae = AE(ae_title = config['PACS']['ae_title'])

# Add a requested presentation context
ae.add_requested_context(PatientRootQueryRetrieveInformationModelMove)

# Add the Storage SCP's supported presentation contexts
ae.supported_contexts = StoragePresentationContexts

# Start our Storage SCP in non-blocking mode, listening on port 11120
ae.ae_title = config['PACS']['ae_title']
scp = ae.start_server((config['PACS']['local_host'], config['PACS'].getint('local_port')), block=False, evt_handlers=handlers)

# Create out identifier (query) dataset
ds = Dataset()
ds.QueryRetrieveLevel = 'IMAGE'
ds.SeriesInstanceUID = '1.2.840.113619.2.80.3834766814.478.1569931317.104.4.1'
ds.StudyInstanceUID = '1.2.826.0.1.3680043.2.146.2.20.2560235.1900228608.0'
ds.PatientID = '2560235'
ds.StudyID = '40464'
ds.StudyDate  = '20191001'
ds.Modality = 'CT'
ds.InstanceNumber = ['1', '100', '200', '289', '298', '300']

# Associate with peer AE at IP 127.0.0.1 and port 11112
assoc = ae.associate(config['PACS']['host'], config['PACS'].getint('port'), ae_title=config['PACS']['ae_called_title'])

if assoc.is_established:
    # Use the C-MOVE service to send the identifier
    responses = assoc.send_c_move(ds, config['PACS']['ae_title'], PatientRootQueryRetrieveInformationModelMove)

    for (status, identifier) in responses:
        if status:
            print('C-MOVE query status: 0x{0:04x}'.format(status.Status))

            # If the status is 'Pending' then `identifier` is the C-MOVE response
            if status.Status in (0xFF00, 0xFF01):
                print(identifier)
        else:
            print('Connection timed out, was aborted or received invalid response')

    # Release the association
    assoc.release()
else:
    print('Association rejected, aborted or never connected')

# Stop our Storage SCP
scp.shutdown()


In [ ]:
[ds.ContentTime for ds in datasets]

In [ ]:
datasets[0].ManufacturerModelName

In [ ]:
# Create out identifier (query) dataset
query_ds = Dataset()
query_ds.QueryRetrieveLevel = 'IMAGE'
query_ds.SeriesInstanceUID = '1.2.840.113619.2.80.3834766814.478.1569931317.104.4.1'
query_ds.StudyInstanceUID = '1.2.826.0.1.3680043.2.146.2.20.2560235.1900228608.0'
query_ds.PatientID = '2560235'
query_ds.StudyID = '40464'
query_ds.StudyDate  = '20191001'
query_ds.Modality = 'CT'
query_ds.InstanceNumber = ['1', '100', '131', '132']

df = find_data(config, query_ds)
df